In [ ]:
# STEP 1 

In [1]:
#####################
# Import packages and config settings
#####################
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd


import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report



pd.set_option('display.max_columns', None)

C:\Users\michi\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\michi\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\michi\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
# STEP 2

In [2]:
#####################
#Load datasets in
#####################
Testing = pd.read_csv(r'C:\Users\michi\Downloads\playground-series-s4e2\test.csv')
Training = pd.read_csv(r'C:\Users\michi\Downloads\playground-series-s4e2\train.csv')
sample = pd.read_csv(r'C:\Users\michi\Downloads\playground-series-s4e2\sample_submission.csv')

In [ ]:
# STEP 3

In [3]:
#####################
#View training data set to begin analyzing how to prep for ML
#####################
display(Training)

# Print the dimensions of the DataFrame
num_rows, num_columns = Training.shape
print(f"DataFrame dimensions: {num_rows} rows x {num_columns} columns")

for column in Training.columns:
    column_type = Training[column].dtype
    unique_values = Training[column].unique()
    print(f"Column: {column}, Type: {column_type}, Unique values: {unique_values}")

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II
20756,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile,Overweight_Level_II


DataFrame dimensions: 20758 rows x 18 columns
Column: id, Type: int64, Unique values: [    0     1     2 ... 20755 20756 20757]
Column: Gender, Type: object, Unique values: ['Male' 'Female']
Column: Age, Type: float64, Unique values: [24.443011 18.       20.952737 ... 25.746113 38.08886  33.852953]
Column: Height, Type: float64, Unique values: [1.699998 1.56     1.71146  ... 1.791366 1.672594 1.536819]
Column: Weight, Type: float64, Unique values: [ 81.66995   57.        50.165754 ... 152.063947  79.5       80.615325]
Column: family_history_with_overweight, Type: object, Unique values: ['yes' 'no']
Column: FAVC, Type: object, Unique values: ['yes' 'no']
Column: FCVC, Type: float64, Unique values: [2.         1.880534   3.         2.679664   2.919751   1.99124
 1.397468   2.636719   1.         1.392665   2.203962   2.971588
 2.668949   1.98989905 2.417635   2.219186   2.919526   2.263245
 2.649406   1.754401   2.303656   2.020785   2.068834   2.689929
 2.979383   2.225731   2.843456   2

In [4]:
#####################
# Hot encode so that it can be read by a ML model - Training
#####################
# Columns to one-hot encode
columns_to_encode = ['Gender','family_history_with_overweight','FAVC','CAEC','SMOKE','SCC','CALC','MTRANS']
Training_encoded = pd.get_dummies(Training, columns=columns_to_encode)

# shift y variable to the end
column_to_shift = 'NObeyesdad'
column = Training_encoded.pop(column_to_shift)
Training_encoded[column_to_shift] = column

#Display dataframe
Training_encoded


,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,SMOKE_no,SMOKE_yes,SCC_no,SCC_yes,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,NObeyesdad
0,0,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,Overweight_Level_II
1,1,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,Normal_Weight
2,2,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,Insufficient_Weight
3,3,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,Obesity_Type_III
4,4,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,25.137087,1.766626,114.187096,2.919584,3.000000,2.151809,1.330519,0.196680,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0,Obesity_Type_II
20754,20754,18.000000,1.710000,50.000000,3.000000,4.000000,1.000000,2.000000,1.000000,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,Insufficient_Weight
20755,20755,20.101026,1.819557,105.580491,2.407817,3.000000,2.000000,1.158040,1.198439,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,Obesity_Type_II
20756,20756,33.852953,1.700000,83.520113,2.671238,1.971472,2.144838,0.000000,0.973834,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,Overweight_Level_II


In [5]:
#####################
#View testing data set to begin analyzing how to prep for ML
#####################
display(Testing)

# Print the dimensions of the DataFrame
num_rows, num_columns = Testing.shape
print(f"DataFrame dimensions: {num_rows} rows x {num_columns} columns")

for column in Testing.columns:
    column_type = Testing[column].dtype
    unique_values = Testing[column].unique()
    print(f"Column: {column}, Type: {column_type}, Unique values: {unique_values}")

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,34593,Male,23.327836,1.721384,78.030383,yes,no,2.813234,3.000000,Sometimes,no,1.000000,no,0.807076,0.778632,Sometimes,Public_Transportation
13836,34594,Female,29.000000,1.590000,62.000000,no,yes,3.000000,3.000000,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation
13837,34595,Female,22.935612,1.585547,44.376637,no,yes,3.000000,2.273740,Frequently,no,2.000000,no,1.949840,1.000000,Sometimes,Public_Transportation
13838,34596,Male,21.000000,1.620000,53.000000,yes,yes,2.000000,3.000000,Sometimes,no,2.000000,no,3.000000,2.000000,no,Public_Transportation


DataFrame dimensions: 13840 rows x 17 columns
Column: id, Type: int64, Unique values: [20758 20759 20760 ... 34595 34596 34597]
Column: Gender, Type: object, Unique values: ['Male' 'Female']
Column: Age, Type: float64, Unique values: [26.899886 21.       26.       ... 39.365005 24.46754  24.451091]
Column: Height, Type: float64, Unique values: [1.848294 1.6      1.643355 ... 1.92     1.518917 1.658393]
Column: Weight, Type: float64, Unique values: [120.644178  66.       111.600553 ... 103.180918  75.306702  85.720788]
Column: family_history_with_overweight, Type: object, Unique values: ['yes' 'no']
Column: FAVC, Type: object, Unique values: ['yes' 'no']
Column: FCVC, Type: float64, Unique values: [2.938616  2.        3.        1.202075  2.061969  2.81646   2.1239
 1.4925    1.851262  2.450784  2.724121  2.633855  2.382705  2.702457
 2.195964  2.037042  1.687569  2.766441  1.617093  2.499108  2.939727
 2.962415  2.442536  2.094184  2.49619   2.041376  2.919526  2.294259
 2.722161  2.971

In [6]:
#####################
# Hot encode so that it can be read by a ML model - Testing
#####################
# Columns to one-hot encode
columns_to_encode = ['Gender','family_history_with_overweight','FAVC','CAEC','SMOKE','SCC','CALC','MTRANS']
Testing_encoded = pd.get_dummies(Testing, columns=columns_to_encode)

#Display dataframe
Testing_encoded


,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,SMOKE_no,SMOKE_yes,SCC_no,SCC_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,20758,26.899886,1.848294,120.644178,2.938616,3.000000,2.825629,0.855400,0.000000,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
1,20759,21.000000,1.600000,66.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
2,20760,26.000000,1.643355,111.600553,3.000000,3.000000,2.621877,0.000000,0.250502,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
3,20761,20.979254,1.553127,103.669116,2.000000,2.977909,2.786417,0.094851,0.000000,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
4,20762,26.000000,1.627396,104.835346,3.000000,3.000000,2.653531,0.000000,0.741069,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,34593,23.327836,1.721384,78.030383,2.813234,3.000000,1.000000,0.807076,0.778632,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
13836,34594,29.000000,1.590000,62.000000,3.000000,3.000000,2.000000,0.000000,0.000000,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
13837,34595,22.935612,1.585547,44.376637,3.000000,2.273740,2.000000,1.949840,1.000000,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
13838,34596,21.000000,1.620000,53.000000,2.000000,3.000000,2.000000,3.000000,2.000000,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0


In [ ]:
# STEP 4

In [7]:
#####################
# Find any missing columns and create a symetry between the datasets 
#####################
differing_columns = Testing_encoded.columns.difference(Training_encoded.columns)

# Display the differing columns
print("Differing columns:", list(differing_columns))

Testing_encoded['CALC_Always'].unique()

Differing columns: ['CALC_Always']


array([0, 1], dtype=uint8)

In [8]:
#####################
# Reconcile the Training dataset to include the CALC_Always Column 
#####################
new_column_name = 'CALC_Always'
after_column = 'SCC_yes'

# Find the index of the column after which to insert the new column
index = Training_encoded.columns.get_loc(after_column) + 1
Training_encoded.insert(index, new_column_name, 0)
Training_encoded

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,SMOKE_no,SMOKE_yes,SCC_no,SCC_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,NObeyesdad
0,0,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Overweight_Level_II
1,1,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,Normal_Weight
2,2,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,Insufficient_Weight
3,3,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Obesity_Type_III
4,4,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,25.137087,1.766626,114.187096,2.919584,3.000000,2.151809,1.330519,0.196680,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Obesity_Type_II
20754,20754,18.000000,1.710000,50.000000,3.000000,4.000000,1.000000,2.000000,1.000000,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Insufficient_Weight
20755,20755,20.101026,1.819557,105.580491,2.407817,3.000000,2.000000,1.158040,1.198439,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,Obesity_Type_II
20756,20756,33.852953,1.700000,83.520113,2.671238,1.971472,2.144838,0.000000,0.973834,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,0,Overweight_Level_II


In [9]:
#####################
# Training and Testing are now symetric besides the y variable
#####################
display(Training_encoded)
display(Testing_encoded)

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,SMOKE_no,SMOKE_yes,SCC_no,SCC_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,NObeyesdad
0,0,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Overweight_Level_II
1,1,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,Normal_Weight
2,2,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,Insufficient_Weight
3,3,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Obesity_Type_III
4,4,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,25.137087,1.766626,114.187096,2.919584,3.000000,2.151809,1.330519,0.196680,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Obesity_Type_II
20754,20754,18.000000,1.710000,50.000000,3.000000,4.000000,1.000000,2.000000,1.000000,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,Insufficient_Weight
20755,20755,20.101026,1.819557,105.580491,2.407817,3.000000,2.000000,1.158040,1.198439,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,Obesity_Type_II
20756,20756,33.852953,1.700000,83.520113,2.671238,1.971472,2.144838,0.000000,0.973834,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,0,Overweight_Level_II


,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,SMOKE_no,SMOKE_yes,SCC_no,SCC_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,20758,26.899886,1.848294,120.644178,2.938616,3.000000,2.825629,0.855400,0.000000,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
1,20759,21.000000,1.600000,66.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
2,20760,26.000000,1.643355,111.600553,3.000000,3.000000,2.621877,0.000000,0.250502,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
3,20761,20.979254,1.553127,103.669116,2.000000,2.977909,2.786417,0.094851,0.000000,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
4,20762,26.000000,1.627396,104.835346,3.000000,3.000000,2.653531,0.000000,0.741069,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,34593,23.327836,1.721384,78.030383,2.813234,3.000000,1.000000,0.807076,0.778632,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
13836,34594,29.000000,1.590000,62.000000,3.000000,3.000000,2.000000,0.000000,0.000000,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
13837,34595,22.935612,1.585547,44.376637,3.000000,2.273740,2.000000,1.949840,1.000000,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
13838,34596,21.000000,1.620000,53.000000,2.000000,3.000000,2.000000,3.000000,2.000000,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0


In [ ]:
# STEP 5

In [ ]:
# First Model

In [40]:
# Specify features (X) and target variable (y)
X = Training_encoded.iloc[:, :-1]
y = Training_encoded['NObeyesdad']

# Encode class labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Define parameter distribution for randomized search
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 1, 2],
    'min_child_weight': [1, 2, 5]
}

# Perform randomized search with cross-validation
random_search = RandomizedSearchCV(xgb_classifier, param_distributions=param_dist, n_iter=20, cv=5, random_state=42)
random_search.fit(X_train, y_train)

# Get the best model from the randomized search
best_xgb_model = random_search.best_estimator_

# Make predictions on the test set
y_pred_xgb = best_xgb_model.predict(X_test)

# Decode predicted labels back to original class names
y_pred_decoded = le.inverse_transform(y_pred_xgb)

# Evaluate the performance of the XGBoost classifier
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy_xgb:.7f}")

# Display classification report for XGBoost
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=le.classes_))


XGBoost Accuracy: 0.9079961
XGBoost Classification Report:
                     precision    recall  f1-score   support

Insufficient_Weight       0.95      0.95      0.95       524
      Normal_Weight       0.89      0.89      0.89       626
     Obesity_Type_I       0.88      0.87      0.88       543
    Obesity_Type_II       0.97      0.97      0.97       657
   Obesity_Type_III       1.00      1.00      1.00       804
 Overweight_Level_I       0.79      0.79      0.79       484
Overweight_Level_II       0.81      0.81      0.81       514

           accuracy                           0.91      4152
          macro avg       0.90      0.90      0.90      4152
       weighted avg       0.91      0.91      0.91      4152



In [43]:
Testing_encoded

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,SMOKE_no,SMOKE_yes,SCC_no,SCC_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,20758,26.899886,1.848294,120.644178,2.938616,3.000000,2.825629,0.855400,0.000000,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
1,20759,21.000000,1.600000,66.000000,2.000000,1.000000,3.000000,1.000000,0.000000,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
2,20760,26.000000,1.643355,111.600553,3.000000,3.000000,2.621877,0.000000,0.250502,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
3,20761,20.979254,1.553127,103.669116,2.000000,2.977909,2.786417,0.094851,0.000000,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
4,20762,26.000000,1.627396,104.835346,3.000000,3.000000,2.653531,0.000000,0.741069,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13835,34593,23.327836,1.721384,78.030383,2.813234,3.000000,1.000000,0.807076,0.778632,0,1,0,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
13836,34594,29.000000,1.590000,62.000000,3.000000,3.000000,2.000000,0.000000,0.000000,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0
13837,34595,22.935612,1.585547,44.376637,3.000000,2.273740,2.000000,1.949840,1.000000,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
13838,34596,21.000000,1.620000,53.000000,2.000000,3.000000,2.000000,3.000000,2.000000,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0


In [46]:
X_test_unseen = Testing_encoded

# Make predictions on the unseen test set
y_pred_unseen = best_xgb_model.predict(X_test_unseen)

# Decode predicted labels back to original class names
y_pred_unseen_decoded = le.inverse_transform(y_pred_unseen)

# Display the predictions for the unseen test set
print("Predictions for Unseen Test Set:")
print(y_pred_unseen_decoded)

Predictions for Unseen Test Set:
['Obesity_Type_II' 'Overweight_Level_I' 'Obesity_Type_III' ...
 'Insufficient_Weight' 'Normal_Weight' 'Obesity_Type_II']


In [49]:
# Convert y_pred_unseen_decoded to a DataFrame
predictions_df = pd.DataFrame({'NObeyesdad': y_pred_unseen_decoded})

selected_columns = ['id'] 
result_df = pd.concat([Testing_encoded[selected_columns], predictions_df], axis=1)
result_df

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Overweight_Level_I
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [50]:
result_df.to_csv('result_dataframe.csv', index=False)

In [ ]:
# Second Model

In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming Training_encoded is your DataFrame with encoded features and target variable
# Make sure your data is properly encoded and doesn't contain any missing values

# Specify features (X) and target variable (y)
X = Training_encoded.iloc[:, :-1]
y = Training_encoded['NObeyesdad']

# Encode class labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize XGBoost classifier with adjusted hyperparameters
xgb_classifier = XGBClassifier(
    n_estimators=300,  # Adjust the number of estimators
    max_depth=7,       # Adjust the maximum depth of trees
    learning_rate=0.1,  # Adjust the learning rate
    subsample=0.9,     # Adjust the subsample ratio
    colsample_bytree=0.9,  # Adjust the ratio of features when constructing each tree
    random_state=42
)

# Train the classifier on the training data
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = xgb_classifier.predict(X_test)

# Evaluate the performance of the XGBoost classifier
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy_xgb:.4f}")

# Display classification report for XGBoost
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))


XGBoost Accuracy: 0.9046
XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       524
           1       0.88      0.90      0.89       626
           2       0.87      0.88      0.88       543
           3       0.98      0.97      0.97       657
           4       0.99      1.00      1.00       804
           5       0.79      0.78      0.78       484
           6       0.79      0.79      0.79       514

    accuracy                           0.90      4152
   macro avg       0.89      0.89      0.89      4152
weighted avg       0.90      0.90      0.90      4152



In [55]:
X_test_unseen = Testing_encoded

# Make predictions on the unseen test set
y_pred_unseen = xgb_classifier.predict(X_test_unseen)

# Decode predicted labels back to original class names
y_pred_unseen_decoded = le.inverse_transform(y_pred_unseen)

# Display the predictions for the unseen test set
print("Predictions for Unseen Test Set:")
print(y_pred_unseen_decoded)

Predictions for Unseen Test Set:
['Obesity_Type_II' 'Overweight_Level_I' 'Obesity_Type_III' ...
 'Insufficient_Weight' 'Normal_Weight' 'Obesity_Type_II']


In [56]:
# Convert y_pred_unseen_decoded to a DataFrame
predictions_df_2 = pd.DataFrame({'NObeyesdad': y_pred_unseen_decoded})

selected_columns = ['id'] 
result_df_2 = pd.concat([Testing_encoded[selected_columns], predictions_df_2], axis=1)
result_df_2

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Overweight_Level_I
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight


In [57]:
result_df_2.to_csv('result_dataframe_2.csv', index=False)